In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
import glob
from IPython.display import clear_output  # ✅ to avoid duplicate display in notebook

# ✅ Tell Dash where to find assets
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "Mammal Carousel"

# 📦 Paths
AQUATIC_PATH = "../assets/mammals/aquatic_mammal"
NONAQUATIC_PARENT = "../assets/mammals"

# 📦 Load species lists
aquatic_species = []
non_aquatic_species = []

try:
    if os.path.exists(AQUATIC_PATH):
        aquatic_species = sorted([
            d for d in os.listdir(AQUATIC_PATH)
            if os.path.isdir(os.path.join(AQUATIC_PATH, d))
        ])
except Exception as e:
    print(f"Error loading aquatic species: {e}")

try:
    if os.path.exists(NONAQUATIC_PARENT):
        non_aquatic_species = sorted([
            d for d in os.listdir(NONAQUATIC_PARENT)
            if os.path.isdir(os.path.join(NONAQUATIC_PARENT, d)) and d != "aquatic_mammal"
        ])
except Exception as e:
    print(f"Error loading non-aquatic species: {e}")

# 📦 Load images
mammal_images = {}

def load_images(folder, key_prefix):
    for species in os.listdir(folder):
        species_path = os.path.join(folder, species)
        if not os.path.isdir(species_path):
            continue
        images = sorted([
            os.path.abspath(f) for f in glob.glob(os.path.join(species_path, "**"), recursive=True)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ])
        if images:
            mammal_images[f"{key_prefix}::{species}"] = images

load_images(AQUATIC_PATH, "aquatic")
load_images(NONAQUATIC_PARENT, "non_aquatic")

# 🔑 Default species
default_species_key = list(mammal_images.keys())[0] if mammal_images else None

# 🎯 Dropdown options
dropdown_options = [
    {"label": k.split("::")[1].replace("_", " ").title(), "value": k}
    for k in mammal_images.keys()
]

# --------------------
# Layout
# --------------------
app.layout = html.Div([
    html.H2("🐻 Mammal Carousel", style={"textAlign": "center", "marginBottom": "25px", "marginTop": "10px"}),

    html.Div([
        html.Label("Select species:"),
        dcc.Dropdown(
            id='species-dropdown',
            options=dropdown_options,
            value=default_species_key,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "40px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# -----------------------------
# Carousel rendering logic
# -----------------------------
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value'),
    prevent_initial_call=False
)
def update_carousel(species_key):
    if not species_key or species_key not in mammal_images:
        return html.P("Please select a species.")

    images = mammal_images[species_key]
    if not images:
        return html.P("No images available for this species.")

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0),
            html.Img(id="carousel-image", src=f"/assets/{relative_path}",
                     style={"maxHeight": "450px", "maxWidth": "400px"}),
            html.Button("➡️", id="next-button", n_clicks=0),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display')
    ])

# -----------------------------
# Navigation buttons
# -----------------------------
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images available"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8899)
